# Update Table Custom Property

This notebook provides an example for updated a table custom property. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:

import requests

API_KEY = "api_key"
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Update table custom property
In this step, we find the table using the title, schema, and database and then update a custom property "CLASSIFICATION" on the table.

In [ ]:
def find_column(title, table, schema, database, *args, **kwargs):
    res = session.get(build_url(f"/table/columns/?title={title}&parent__title={table}&parent__parent__title={schema}&parent__parent__parent__title={database}"))
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None


def set_column_properties(unique_id, properties):
    session.patch(
        build_url(f"/table/columns/{unique_id}/"),
        json=dict(
            properties=properties,
        ),
    )


columns = [{
    "title": "payment_id",
    "table": "stg_payments",
    "schema": "lightdash",
    "database": "secodaredshift"
}]
property_name = "CLASSIFICATION"
new_value = "Test"

for c in columns:
    column = find_column(**c)
    if column:
        print(f"Found column: {column.get('title')}")
        properties = column.get("properties", {})
        if not properties.get("custom", None):
            properties["custom"] = []
        
        print("PROPERTIES: ", properties["custom"])

        if property_name in properties:
            properties[property_name] = new_value
        else:
            properties["custom"][property_name] = new_value
        set_column_properties(column.get("id"), properties)
